In [291]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

#returns all upcoming games and average money lines given the sports vegasinsider page
def get_odds(url):
    soup = bs(requests.get(url).text, 'html.parser')
    df = pd.DataFrame(columns=['Away Team', 'Away Odds', 'Home Team', 'Home Odds'])
    games = soup.find('table', {'class':'frodds-data-tbl'}).find_all('tr')
    for game in games:
        away_team, home_team = [tag.text for tag in game.find_all('a')[:2]]
        odds = re.findall('<br/>([\+,-]\d{3,5})<br/>([\+,-]\d{0,5})', str(game))
        if len(odds) > 0:
            away_odds, home_odds = map(list, zip(*odds))
            away_odds = int(sum([int(o) for o in away_odds])/len(away_odds))
            home_odds = int(sum([int(o) for o in home_odds])/len(home_odds))
            row = {'Away Team':away_team, 'Away Odds':away_odds, 'Home Team':home_team, 'Home Odds':home_odds}
            df = df.append(row, ignore_index=True)
    return df

#hard coded the urls so the user doesnt need to guess
def nfl():
    url = 'https://www.vegasinsider.com/nfl/odds/las-vegas/money/'
    return get_odds(url)

def ncaaf():
    url = 'https://www.vegasinsider.com/college-basketball/odds/las-vegas/money/'
    return get_odds(url)

def ncaab():
    url = 'https://www.vegasinsider.com/college-basketball/odds/las-vegas/money/'
    return get_odds(url)

def nba():
    url = 'https://www.vegasinsider.com/nba/odds/las-vegas/money/'
    return get_odds(url)

def nhl():
    url = 'https://www.vegasinsider.com/nhl/odds/las-vegas/'
    return get_odds(url)

def mlb():
    url = 'https://www.vegasinsider.com/mlb/odds/las-vegas/run/'
    return get_odds(url)

#returns the specific money line given a team and their table
def lookup(team, df):
    for team_idx in [0,2]:
            col = df.columns[team_idx]
            if team in list(df[col]):
                return df[df[col] == team].iat[0,team_idx+1]
    return 0

#returns the calculated payout given the line and wager
def payout(line, wager):
    if line > 0:
        return wager*line/100
    if line < 0:
        return -wager*100/line
    return 0

#returns the payout if betting wager on team
def gamble(team, wager):
    for df in [nfl(), ncaaf(), ncaab(), nba(), nhl(), mlb()]:
        pay = payout(lookup(team, df), wager)
        if pay > 0:
            return round(pay,2)